In [12]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.3.0 spark-nlp==4.2.8
! pip install Unidecode
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00


In [2]:
import string
import unidecode
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [4]:
spark = sparknlp.start(gpu=True)
model_hatexplain = "bert_sequence_classifier_hatexplain"
model_emotion = "bert_sequence_classifier_emotion"

text_hatexplain = ["""Go back to your country Muslims!"""]

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = unidecode.unidecode(text)
    return text

text_hatexplain = [preprocess_text(text) for text in text_hatexplain]

In [6]:
model_dict = {model_hatexplain: text_hatexplain, model_emotion: text_hatexplain}

In [7]:
def run_pipeline(model, text, results):  
  document_assembler = DocumentAssembler() \
      .setInputCol('text') \
      .setOutputCol('document')

  tokenizer = Tokenizer() \
      .setInputCols(['document']) \
      .setOutputCol('token')

  sequenceClassifier = BertForSequenceClassification\
        .pretrained(model, 'en') \
        .setInputCols(['token', 'document']) \
        .setOutputCol('pred_class')

  pipeline = Pipeline(
      stages=[
          document_assembler, 
          tokenizer, 
          sequenceClassifier])

  df = spark.createDataFrame(text, StringType()).toDF("text")
  results[model]=(pipeline.fit(df).transform(df))

In [8]:
results = {}
for model, text in zip(model_dict.keys(),model_dict.values()):
  run_pipeline(model, text, results)

bert_sequence_classifier_hatexplain download started this may take some time.
Approximate size to download 391.1 MB
[OK!]
bert_sequence_classifier_emotion download started this may take some time.
Approximate size to download 391.1 MB
[OK!]


In [9]:
model_dfs = {}
for model_name, result in zip(results.keys(),results.values()):  
  res = result.select(F.explode(F.arrays_zip(result.document.result, 
                                             result.pred_class.result,
                                             result.pred_class.metadata)).alias("col"))\
              .select(F.expr("col['1']").alias("prediction"),
                      F.expr("col['2']").alias("confidence"),
                      F.expr("col['0']").alias("sentence"))
  if res.count() > 0:
        udf_func = F.udf(lambda x, y:  x[str(y)])
        print("\n",model_name,"\n") 
        res.withColumn('confidence', udf_func(res.confidence, res.prediction)).show(truncate=False)
        # add the resulting data frame to the dictionary
        model_dfs[model_name + "_df"] = res
        print("\n**********************************\n")


 bert_sequence_classifier_hatexplain 

+----------+----------+--------------------------------+
|prediction|confidence|sentence                        |
+----------+----------+--------------------------------+
|offensive |0.41789165|go back to your country muslims!|
+----------+----------+--------------------------------+


**********************************


 bert_sequence_classifier_emotion 

+----------+----------+--------------------------------+
|prediction|confidence|sentence                        |
+----------+----------+--------------------------------+
|joy       |0.53935915|go back to your country muslims!|
+----------+----------+--------------------------------+


**********************************



In [10]:
hate_df = model_dfs["bert_sequence_classifier_hatexplain_df"]
emotion_df = model_dfs["bert_sequence_classifier_emotion_df"]

hate_df = hate_df.withColumnRenamed('prediction', 'speech')
hate_df = hate_df.drop("confidence")
emotion_df = emotion_df.withColumnRenamed('prediction', 'emotion')
emotion_df = emotion_df.drop("confidence")

joined_df = hate_df.join(emotion_df, "sentence")
joined_df.show()

+--------------------+---------+-------+
|            sentence|   speech|emotion|
+--------------------+---------+-------+
|go back to your c...|offensive|    joy|
+--------------------+---------+-------+



In [14]:
from pyspark.sql.functions import first, col
input = joined_df.select(first(col(joined_df.columns[0]))).collect()[0][0]

print(input)

go back to your country muslims!


In [11]:
from pyspark.sql.functions import when, concat_ws

def add_counterspeech_column(df):
    df_with_counterspeech = df.withColumn(
        "counterspeech",
        when((df.speech == "hate speech") & (df.emotion == "surprise"), "I'm surprised to hear that kind of language. Let's strive to use our words to create positive change instead.")
        .when((df.speech == "hate speech") & (df.emotion == "joy"), "There's nothing to celebrate about hate speech or offensive language. Let's focus on kindness and respect instead.")
        .when((df.speech == "hate speech") & (df.emotion == "anger"), "Your anger is misplaced. It's important to focus on the issue, not attack individuals or groups.")
        .when((df.speech == "hate speech") & (df.emotion == "love"), "Love is about acceptance and kindness. Let's work towards building a world where everyone feels valued and respected.")
        .when((df.speech == "hate speech") & (df.emotion == "fear"), "It's important to address real issues, but spreading fear and hate only makes things worse.")
        .when((df.speech == "hate speech") & (df.emotion == "sadness"), "It's sad that we live in a world where hate speech and offensive language exist. Let's work together to change that.")
        .when((df.speech == "offensive") & (df.emotion == "surprise"), "We should use our words to build bridges, not walls.")
        .when((df.speech == "offensive") & (df.emotion == "joy"), "It's important to recognize the harm that words can cause. Let's work towards building a more inclusive and respectful society.")
        .when((df.speech == "offensive") & (df.emotion == "anger"), "I understand you're angry, but your words are hurtful and not productive.")
        .when((df.speech == "offensive") & (df.emotion == "love"), "We should use our words to uplift and support each other, not tear each other down.")
        .when((df.speech == "offensive") & (df.emotion == "fear"), "We should try to understand each other instead of resorting to fear-mongering.")
        .when((df.speech == "offensive") & (df.emotion == "sadness"), "Words can hurt, and it's important to recognize that. Let's work towards creating a society where everyone feels safe and respected.")
        .otherwise("*")
    )
    return df_with_counterspeech

joined_df = add_counterspeech_column(joined_df)
joined_df.show()

joined_df = joined_df.filter(joined_df.counterspeech != '*')
joined_df.show()

+--------------------+---------+-------+--------------------+
|            sentence|   speech|emotion|       counterspeech|
+--------------------+---------+-------+--------------------+
|go back to your c...|offensive|    joy|It's important to...|
+--------------------+---------+-------+--------------------+

+--------------------+---------+-------+--------------------+
|            sentence|   speech|emotion|       counterspeech|
+--------------------+---------+-------+--------------------+
|go back to your c...|offensive|    joy|It's important to...|
+--------------------+---------+-------+--------------------+



In [24]:
input_speech = joined_df.select(first(col(joined_df.columns[1]))).collect()[0][0]
print(input_speech)

if (input_speech == "hate speech" or "offensive"):
  print("Process Counter Speech")
else:
  print("Speech is normal!")

offensive
Process Counter Speech


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [15]:
# from transformers import GPT3Tokenizer, GPT3Model

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

input_sequence = input

input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

generated_output = model.generate(
    input_ids=input_ids,
    max_length=90,
    pad_token_id=tokenizer.eos_token_id,
)

print(tokenizer.decode(generated_output[0], skip_special_tokens=True))

go back to your country muslims!

The following is a list of the most popular muslims in the world.

1. Mohammed Ali

Mohammed Ali is the most popular muslim in the world. He is the most famous muslim in the world. He is the most famous muslim in the world. He is the most famous muslim in the world. He is the most famous muslim in the


In [16]:
from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data["conan"]:
                self.X.append("<startofstring> "+i['hateSpeech']+" obsiously wrong because <bot>: " +i['counterSpeech']+" <endofstring>")

        print(self.X[0])

        # self.X = self.X[]  
        self.X_encoded = tokenizer(self.X,max_length=60, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [19]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):

    epochs = 15
    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state2.pt")
        print(infer("I want to go back to school and get an education but i am poor and colleges are kike rip offs i am not satisfied with where i am in life and honestly have no clue where or how to start pushing forward"))

def infer(inp):
    inp = "<startofstring> "+inp+" obsiously wrong because <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a, max_new_tokens=60)#, pad_token_id=tokenizer.eos_token_id
    output = tokenizer.decode(output[0])
    output.split 
    return output



In [25]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

chatData = ChatData("./hate_speech_english.json", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)

model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)

<startofstring> According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful. obsiously wrong because <bot>: To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith. <endofstring>
training .... 


  7%|▋         | 1/15 [01:03<14:45, 63.22s/it]

<startofstring> I want to go back to school and get an education but i am poor and colleges are kike rip offs i am not satisfied with where i am in life and honestly have no clue where or how to start pushing forward obsiously wrong because <bot>: iously wrong because <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 13%|█▎        | 2/15 [02:06<13:40, 63.09s/it]

<startofstring> I want to go back to school and get an education but i am poor and colleges are kike rip offs i am not satisfied with where i am in life and honestly have no clue where or how to start pushing forward obsiously wrong because <bot>: I <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


...

<startofstring> I want to go back to school and get an education but i am poor and colleges are kike rip offs i am not satisfied with where i am in life and honestly have no clue where or how to start pushing forward obsiously wrong because <bot>: What type of whims are you talking about? Laws, education, etc. Laws are all about death and genocide. <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 93%|█████████▎| 14/15 [14:47<01:03, 63.32s/it]

<startofstring> I want to go back to school and get an education but i am poor and colleges are kike rip offs i am not satisfied with where i am in life and honestly have no clue where or how to start pushing forward obsiously wrong because <bot>: Evidence? <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 15/15 [15:50<00:00, 63.39s/it]

<startofstring> I want to go back to school and get an education but i am poor and colleges are kike rip offs i am not satisfied with where i am in life and honestly have no clue where or how to start pushing forward obsiously wrong because <bot>: What is your point? This is not about the Muslim community. This is a motivation for stupid sexual psychopaths and for criminals. obsiously wrong because <bot>: I do not think it is a religion of the criminals who contaminate the world. I wonder why would make this a religion so many? Why


In [26]:
print("infer from model : ")
while True:
  inp = input
  print(infer(inp))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


infer from model : 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> go back to your country muslims! obsiously wrong because <bot>: What happened, my friend? Do you have any proofs for this? I did not have them. So I am sceptical. <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> go back to your country muslims! obsiously wrong because <bot>: Why would you say that? I cannot you! <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> go back to your country muslims! obsiously wrong because <bot>: Why would you say that? I would never say this. (insert link to first/second world war Muslim invasion) <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


KeyboardInterrupt: ignored